In [ ]:
import pandas as pd
import os
from IPython.display import display

date_list = ['2019-20', '2020-21', '2021-22', '2022-23', '2023-24']
player_names = ['Jayson Tatum', 'Devin Booker']
player_dataframes = {}  # Dictionary to store each player's DataFrame

for player in player_names:
    player_combined_df = pd.DataFrame()  # Initialize a DataFrame for the current player
    
    for date in date_list:
        path = fr"C:\Users\mandy\Documents\Github\web_scraping_sports\nba_ph_csv\season_{date}\all_quarters\{player}_content.csv"
        
        if os.path.exists(path):
            season_df = pd.read_csv(path)
            print(f"Found file: {path}")

            season_df['season'] = date  # Add the season column
            player_combined_df = pd.concat([player_combined_df, season_df], ignore_index=True)
        else:
            print(f"{player} {date} not found")
    
    # Add the player's DataFrame to the dictionary
    player_dataframes[player] = player_combined_df

# Example: Access each player's DataFrame
for player, df in player_dataframes.items():
    print(f"Data for {player}:")
    display(df)  # Display the first few rows of each player's DataFrame


In [ ]:
import pandas as pd
import os

date_list = ['2019-20', '2020-21', '2021-22', '2022-23', '2023-24']
player_name = ['Jayson Tatum', 'Devin Booker']
player_dataframes = {}  # Dictionary to store each player's DataFrame

for player in player_name:
    single_player_df = pd.DataFrame()  # Initialize a DataFrame for the current player
    defense_df = pd.DataFrame()  # Initialize a DataFrame for the current player's defense
    
    for date in date_list:
        path = fr"C:\Users\mandy\Documents\Github\web_scraping_sports\historic_player_data\nba_ph_csv\season_{date}\all_quarters\{player}_content.csv"
        path_defense = fr'C:\Users\mandy\Documents\Github\web_scraping_sports\historic_defense_data\nba_dh_csv\defense_csv_{date}\all_quarter_defense_content.csv'

        if os.path.exists(path) and os.path.exists(path_defense):
            #player data
            season_df_player = pd.read_csv(path)
            season_df_player['season'] = date
            single_player_df = pd.concat([single_player_df, season_df_player], ignore_index=True)
            #defense data
            defense_df_season = pd.read_csv(path_defense)
            defense_df_season['season'] = date
            defense_df = pd.concat([defense_df, defense_df_season], ignore_index=True)
            
        else:
            print(f'{player} {date} not found')

    # Add the player's DataFrame to the dictionary
    player_dataframes[player] = single_player_df
    #print(defense_df[defense_df['season'] < '2022-23'])
#player_dataframes['Devin Booker'].head(20)
defense_df

#     jayson_df = player_dataframes['Jayson Tatum']
# jayson_df.head(20)

# # for player, df in player_dataframes.items():
# #     print(f"Data for {player}:")
    
#     for df in player_dataframes.values():
#         print(df.head())
#      # Display the first few rows of each player's DataFrame


In [15]:
import pandas as pd
import os
from IPython.display import display
import numpy as np
from pathlib import Path



date_list = ['2019-20', '2020-21', '2021-22', '2022-23', '2023-24']
player_name = ['Shai Gilgeous-Alexander', 'Cason Wallace', 'Alex Caruso', 'Jalen Williams', 'Isaiah Hartenstein']
player_dataframes = {}  # Dictionary to store each player's DataFrame

for player in player_name:
    single_player_df = pd.DataFrame()  # Initialize a DataFrame for the current player
    defense_df = pd.DataFrame()  # Initialize a DataFrame for the current player's defense
    
    for date in date_list:
        notebook_dir = Path().resolve()
        data_path = notebook_dir / f"../../historic_player_data/nba_ph_csv/season_{date}/all_quarters/{player}_content.csv"
        data_path_defense = notebook_dir / f"../../historic_defense_data/nba_dh_csv/defense_csv_{date}/all_quarter_defense_content.csv"

        path = data_path.resolve()
        path_defense = data_path_defense.resolve()
        if os.path.exists(path) and os.path.exists(path_defense):
            #player data
            season_df_player = pd.read_csv(path)
            season_df_player['season'] = date
            single_player_df = pd.concat([single_player_df, season_df_player], ignore_index=True)
            #defense data
            defense_df_season = pd.read_csv(path_defense)
            defense_df_season['season_defense'] = date
            defense_df = pd.concat([defense_df, defense_df_season], ignore_index=True)
            
        else:
            print(f'{date} not found either for {player} or defense')
    
    # Add the player's DataFrame to the dictionary 
    pd.set_option('display.max_rows', 1000)  # Maximum number of rows to display
    pd.set_option('display.max_columns', None)  # Show all columns
    pd.set_option('display.width', 1000)  # Adjust column width for better readability
    
    player_dataframes[player] = single_player_df

    df_defense = defense_df[['TEAM', 'DefRtg', 'season_defense']]

    merged_df = pd.merge(player_dataframes[player], df_defense, how='inner', left_on=['Away','season'], right_on=['TEAM','season_defense']).reset_index(drop=True)
    merged_df = merged_df.sort_values(by="Date")

    display(merged_df.head(10))

    train_data = merged_df[merged_df['Date'] < '2023-01-01']  # Training: First 4 seasons
    test_data = merged_df[merged_df['Date'] >= '2023-01-01']  # Testing: Most recent season

    # Select features and target
    what_train = train_data[['MIN', 'FGA', 'FG%', 'FTA', 'FGM', 'FTM', 'DefRtg']]
    what_test = test_data[['MIN', 'FGA', 'FG%', 'FTA', 'FGM', 'FTM', 'DefRtg']]

    # X_train = train_data[['MIN', 'FGA', 'FG%', 'FTA', 'DefRtg']]
    X_train = what_train
    y_train = train_data['PTS']
    # X_test = test_data[['MIN', 'FGA', 'FG%', 'FTA', 'DefRtg']]
    X_test = what_test
    y_test = test_data['PTS']

    try:
        # Train model
        from sklearn.linear_model import LinearRegression
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Test model
        predictions = model.predict(X_test)   

        # Evaluate
        from sklearn.metrics import mean_absolute_error, mean_squared_error
        mae = mean_absolute_error(y_test, predictions)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))

        # Compare predictions with actual points
        predicted_vs_actual = pd.DataFrame({
        "Date": test_data['Date'],  
        "Matchup": test_data['Matchup'],  # or adjust based on how teams are stored
        "Predicted Points": predictions.round(1), # Adjust for underestimation we want underestimation,
        "Actual Points": y_test}).reset_index(drop=True)

        display(predicted_vs_actual.head(5))

        print(f"{player} MAE: {mae:.2f}, RMSE: {rmse:.2f}")

        # Combine train and test data for final training
        X_full = pd.concat([X_train, X_test], ignore_index=True)
        y_full = pd.concat([y_train, y_test], ignore_index=True)

        # Train the model on all historical data
        model.fit(X_full, y_full)

        recent_games = merged_df[-5:]  # Last 5 games
        avg_min = recent_games['MIN'].mean()
        avg_fga = recent_games['FGA'].mean()
        avg_fg = recent_games['FG%'].mean()
        avg_fta = recent_games['FTA'].mean()


        X_future = pd.DataFrame({
        'MIN': avg_min,         # Projected minutes played
        'FGA': [14.76],         # Projected field goal attempts
        'FG%': [58.631],         # Projected field goal percentage
        'FTA': [5.73],          # Projected free throw attempts
        'DefRtg': [116.3]     # Opponent defensive rating
    })
    
    except Exception as e:
        print(f"{e}")

    # predicted_points = model.predict(X_future)
    # predicted_points = predicted_points.round(1) - 5
    # print(f"{player} Predicted Points: {predicted_points[0]:.1f}")

    # display(X_future)




,Date,Matchup,Team,Away,Home/Away_game,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,season,TEAM,DefRtg,season_defense
49,2019-12-06,OKC vs. MIN,OKC,MIN,Home,W,41.32,29,8,13,61.5,1,1,100.0,12,12,100.0,1,4,5,2,1,0,3,2,9,2019-20,MIN,111.6,2019-20
48,2019-12-08,OKC @ POR,OKC,POR,Away,W,38.15,21,5,14,35.7,2,4,50.0,9,12,75.0,0,9,9,3,1,0,1,2,6,2019-20,POR,114.3,2019-20
47,2019-12-09,OKC @ UTA,OKC,UTA,Away,W,33.28,20,7,20,35.0,2,5,40.0,4,6,66.7,0,6,6,3,2,0,1,2,5,2019-20,UTA,109.3,2019-20
46,2019-12-11,OKC @ SAC,OKC,SAC,Away,L,35.33,14,4,11,36.4,0,1,0.0,6,8,75.0,0,4,4,4,1,1,3,3,-3,2019-20,SAC,111.4,2019-20
45,2019-12-14,OKC @ DEN,OKC,DEN,Away,L,37.44,16,7,13,53.8,0,1,0.0,2,2,100.0,0,4,4,3,0,0,2,4,-5,2019-20,DEN,110.4,2019-20
44,2019-12-16,OKC vs. CHI,OKC,CHI,Home,W,34.24,7,1,3,33.3,0,1,0.0,5,8,62.5,0,3,3,2,1,0,5,0,-1,2019-20,CHI,108.9,2019-20
43,2019-12-18,OKC vs. MEM,OKC,MEM,Home,W,38.42,20,7,16,43.8,0,3,0.0,6,6,100.0,2,7,9,1,2,2,1,1,14,2019-20,MEM,109.7,2019-20
42,2019-12-20,OKC vs. PHX,OKC,PHX,Home,W,36.06,32,13,18,72.2,2,2,100.0,4,5,80.0,0,4,4,3,1,0,0,2,11,2019-20,PHX,110.8,2019-20
41,2019-12-22,OKC vs. LAC,OKC,LAC,Home,W,34.29,32,12,25,48.0,2,3,66.7,6,6,100.0,0,3,3,5,2,0,2,2,16,2019-20,LAC,106.9,2019-20
40,2019-12-26,OKC vs. MEM,OKC,MEM,Home,L,37.08,21,8,19,42.1,1,6,16.7,4,4,100.0,0,3,3,1,3,0,1,0,-9,2019-20,MEM,109.7,2019-20


,Date,Matchup,Predicted Points,Actual Points
0,2023-01-04,OKC @ ORL,33.9,33
1,2023-01-06,OKC vs. WAS,30.7,30
2,2023-01-08,OKC vs. DAL,34.5,33
3,2023-01-10,OKC @ MIA,25.7,26
4,2023-01-12,OKC @ PHI,37.1,37


Shai Gilgeous-Alexander MAE: 0.77, RMSE: 1.01
2019-20 not found either for Cason Wallace or defense
2020-21 not found either for Cason Wallace or defense
2021-22 not found either for Cason Wallace or defense
2022-23 not found either for Cason Wallace or defense


,Date,Matchup,Team,Away,Home/Away_game,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,season,TEAM,DefRtg,season_defense
49,2024-01-03,OKC @ ATL,OKC,ATL,Away,L,19.54,15,5,9,55.6,3,7,42.9,2,2,100.0,0,1,1,2,1,0,0,2,-16,2023-24,ATL,118.4,2023-24
48,2024-01-05,OKC @ BKN,OKC,BKN,Away,L,23.27,8,3,7,42.9,2,4,50.0,0,0,0.0,1,1,2,0,1,0,0,1,-5,2023-24,BKN,115.4,2023-24
47,2024-01-08,OKC @ WAS,OKC,WAS,Away,W,24.00,5,2,7,28.6,1,4,25.0,0,0,0.0,1,1,2,2,2,3,1,1,-4,2023-24,WAS,118.9,2023-24
46,2024-01-10,OKC @ MIA,OKC,MIA,Away,W,18.14,8,3,5,60.0,2,4,50.0,0,0,0.0,1,1,2,1,0,0,1,2,9,2023-24,MIA,111.5,2023-24
45,2024-01-11,OKC vs. POR,OKC,POR,Home,W,17.25,4,1,5,20.0,0,3,0.0,2,2,100.0,1,5,6,2,0,0,0,0,17,2023-24,POR,116.6,2023-24
44,2024-01-13,OKC vs. ORL,OKC,ORL,Home,W,19.45,1,0,5,0.0,0,4,0.0,1,2,50.0,2,1,3,0,0,0,0,3,5,2023-24,ORL,110.8,2023-24
43,2024-01-15,OKC @ LAL,OKC,LAL,Away,L,19.38,5,2,6,33.3,1,4,25.0,0,0,0.0,0,0,0,1,2,1,1,0,-7,2023-24,LAL,114.8,2023-24
42,2024-01-16,OKC @ LAC,OKC,LAC,Away,L,14.55,5,2,4,50.0,1,2,50.0,0,0,0.0,2,0,2,1,1,1,0,1,-1,2023-24,LAC,114.6,2023-24
41,2024-01-18,OKC @ UTA,OKC,UTA,Away,W,32.29,16,6,7,85.7,4,5,80.0,0,0,0.0,2,2,4,1,1,1,1,3,-1,2023-24,UTA,119.6,2023-24
40,2024-01-20,OKC @ MIN,OKC,MIN,Away,W,12.33,0,0,1,0.0,0,0,0.0,0,0,0.0,1,2,3,0,1,0,1,2,-15,2023-24,MIN,108.4,2023-24


Found array with 0 sample(s) (shape=(0, 7)) while a minimum of 1 is required by LinearRegression.


...

,Date,Matchup,Predicted Points,Actual Points
0,2023-01-03,OKC vs. BOS,24.9,21
1,2023-01-04,OKC @ ORL,5.6,4
2,2023-01-06,OKC vs. WAS,17.4,18
3,2023-01-08,OKC vs. DAL,9.5,10
4,2023-01-10,OKC @ MIA,20.7,12


Jalen Williams MAE: 4.26, RMSE: 5.41


,Date,Matchup,Team,Away,Home/Away_game,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,season,TEAM,DefRtg,season_defense
22,2019-11-15,HOU vs. IND,HOU,IND,Home,W,13.14,4,2,2,100.0,0,0,0.0,0,0,0.0,0,0,0,0,0,1,0,2,-1,2019-20,IND,107.5,2019-20
21,2019-11-16,HOU @ MIN,HOU,MIN,Away,W,30.37,6,1,2,50.0,0,0,0.0,4,4,100.0,5,11,16,0,0,0,2,5,26,2019-20,MIN,111.6,2019-20
20,2019-11-18,HOU vs. POR,HOU,POR,Home,W,12.51,9,3,3,100.0,0,0,0.0,3,3,100.0,0,1,1,0,0,2,2,1,-6,2019-20,POR,114.3,2019-20
19,2019-11-20,HOU @ DEN,HOU,DEN,Away,L,4.21,2,1,1,100.0,0,0,0.0,0,0,0.0,0,0,0,1,1,0,0,1,4,2019-20,DEN,110.4,2019-20
18,2019-11-24,HOU vs. DAL,HOU,DAL,Home,L,1.48,2,1,1,100.0,0,0,0.0,0,0,0.0,0,1,1,0,0,0,0,0,5,2019-20,DAL,111.2,2019-20
17,2019-11-27,HOU vs. MIA,HOU,MIA,Home,W,10.24,1,0,1,0.0,0,0,0.0,1,2,50.0,0,2,2,1,0,0,0,2,-1,2019-20,MIA,109.3,2019-20
16,2019-11-30,HOU vs. ATL,HOU,ATL,Home,W,20.47,7,3,5,60.0,0,1,0.0,1,2,50.0,3,2,5,2,1,1,2,4,15,2019-20,ATL,114.4,2019-20
15,2019-12-07,HOU vs. PHX,HOU,PHX,Home,W,2.24,0,0,0,0.0,0,0,0.0,0,0,0.0,0,1,1,0,0,0,0,1,-7,2019-20,PHX,110.8,2019-20
14,2019-12-13,HOU @ ORL,HOU,ORL,Away,W,6.38,0,0,1,0.0,0,0,0.0,0,0,0.0,0,0,0,1,1,0,0,1,-2,2019-20,ORL,109.2,2019-20
13,2019-12-28,HOU vs. BKN,HOU,BKN,Home,W,24.22,9,4,7,57.1,0,0,0.0,1,2,50.0,6,7,13,1,1,0,2,4,-4,2019-20,BKN,109.2,2019-20


,Date,Matchup,Predicted Points,Actual Points
0,2023-01-02,NYK vs. PHX,3.1,3
1,2023-01-04,NYK vs. SAS,8.2,8
2,2023-01-06,NYK @ TOR,2.2,2
3,2023-01-09,NYK vs. MIL,0.3,0
4,2023-01-11,NYK vs. IND,-0.1,0


Isaiah Hartenstein MAE: 0.26, RMSE: 0.30
